In [7]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key_train = 'Forecasting-Flow-Traffic/train.csv'
file_key_test = 'Forecasting-Flow-Traffic/test.csv'

bucket_object_train = bucket.Object(file_key_train)
file_object_train = bucket_object_train.get()
file_content_stream_train = file_object_train.get('Body')

bucket_object_test = bucket.Object(file_key_test)
file_object_test = bucket_object_test.get()
file_content_stream_test = file_object_test.get('Body')

## Reading the csv file
train = pd.read_csv(file_content_stream_train)
test = pd.read_csv(file_content_stream_test)

## Puting time in the right format 
train['time'] = pd.to_datetime(train['time'], format = '%Y-%m-%d %H:%M:%S')
test['time'] = pd.to_datetime(test['time'], format = '%Y-%m-%d %H:%M:%S')

## Quick Distribution Comparisons

In [9]:
train['direction'].value_counts()

SB    156708
NB    156708
EB    156708
WB    143649
SW     91413
NE     91413
SE     26118
NW     26118
Name: direction, dtype: int64

In [10]:
test['direction'].value_counts()

EB    432
NB    432
SB    432
WB    396
SW    252
NE    252
NW     72
SE     72
Name: direction, dtype: int64

In [12]:
train['time']

0        1991-04-01 00:00:00
1        1991-04-01 00:00:00
2        1991-04-01 00:00:00
3        1991-04-01 00:00:00
4        1991-04-01 00:00:00
                 ...        
848830   1991-09-30 11:40:00
848831   1991-09-30 11:40:00
848832   1991-09-30 11:40:00
848833   1991-09-30 11:40:00
848834   1991-09-30 11:40:00
Name: time, Length: 848835, dtype: datetime64[ns]

In [4]:
test.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB
